In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
# Load the Iris dataset
data = load_iris()
X = data.data
y = (data.target != 0).astype(int)  # Convert to binary classification problem

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [5]:
# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self, input_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


In [6]:
# Create an instance of the neural network
input_dim = X_train.shape[1]
model = SimpleNN(input_dim)

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [9]:
# Training loop
epochs = 1000
for epoch in range(epochs):
    optimizer.zero_grad()  # Zero the gradients
    outputs = model(X_train)  # Forward pass
    loss = criterion(outputs, y_train.view(-1, 1))  # Calculate the loss
    loss.backward()  # Backpropagation
    optimizer.step()  # Update weights

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}')

# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    predicted = (test_outputs >= 0.8).float()
    accuracy = accuracy_score(y_test, predicted.numpy())
    print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch [100/1000], Loss: 3.432252196944319e-05
Epoch [200/1000], Loss: 2.9990918847033754e-05
Epoch [300/1000], Loss: 2.6405206881463528e-05
Epoch [400/1000], Loss: 2.3427332052960992e-05
Epoch [500/1000], Loss: 2.089759254886303e-05
Epoch [600/1000], Loss: 1.8730217561824247e-05
Epoch [700/1000], Loss: 1.6860418327269144e-05
Epoch [800/1000], Loss: 1.5234689271892421e-05
Epoch [900/1000], Loss: 1.3809984920953866e-05
Epoch [1000/1000], Loss: 1.256110226677265e-05
Test Accuracy: 100.00%
